**Installing latest pickle to unserialize our data from database**

In [3]:
!pip install pickle5


**Adding needed imports**

In [4]:
from google.colab import drive
import os
import  pickle5 as pickle
from HandReading import HandReading
from Imu import Imu
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, LSTM
from sklearn.metrics import accuracy_score as acc
from tensorflow.keras.utils import plot_model
from sklearn.metrics import classification_report
drive.mount('/content/drive')

Mounted at /content/drive


**Creating dictionary mappers**


In [5]:
dict_word_id = {}
dict_id_word = {}

list_word = os.listdir('/content/drive/MyDrive/database') 
for j in range(len(list_word)) : 
  dict_word_id[list_word[j]] = j
  dict_id_word[j] = list_word[j]
print(dict_word_id)
print(dict_id_word)

{'23': 0, 'lucas': 1, 'oi': 2, 'silencio': 3, 'tchau': 4, 'tudo-bem': 5, 'qual-a-sua-idade': 6, 'prazer-em-te-conhecer': 7, 'meu-nome-e': 8, 'abdullah': 9, 'qual-seu-nome': 10}
{0: '23', 1: 'lucas', 2: 'oi', 3: 'silencio', 4: 'tchau', 5: 'tudo-bem', 6: 'qual-a-sua-idade', 7: 'prazer-em-te-conhecer', 8: 'meu-nome-e', 9: 'abdullah', 10: 'qual-seu-nome'}


**Preparing input and labels data**

In [6]:
input_data, output = [], []

list_word = os.listdir('/content/drive/MyDrive/database') 

for word in list_word:
  list_pickle_file = os.listdir(f'/content/drive/MyDrive/database/{word}')
  k = 0 
  for pickle_file in list_pickle_file:
    k+=1
    with open(f'/content/drive/MyDrive/database/{word}/{pickle_file}', 'rb') as input:
      readings = pickle.load(input)
      for reading in readings:
        finger = []
        for i in range(5):
          finger.append(np.float16(reading.imus[i].accel[0]/255.0))
          finger.append(np.float16(reading.imus[i].accel[1]/255.0))
          finger.append(np.float16(reading.imus[i].accel[2]/255.0))
          finger.append(np.float16(reading.imus[i].gyro[0]/255.0))
          finger.append(np.float16(reading.imus[i].gyro[1]/255.0))
          finger.append(np.float16(reading.imus[i].gyro[2]/255.0))
          pass
      
        input_data.append(finger)
        
        out = np.zeros(11)
        out[dict_word_id[word]] = 1.0
        output.append(out)

input_data = np.array(input_data)
input_data = input_data.reshape((input_data.shape[0], input_data.shape[1], 1))
output = np.array(output)

**Creating neural network model**

In [31]:
model = Sequential()
model.add(LSTM(units=50, input_shape=input_data[0].shape))
model.add(Dense(11, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 50)                10400     
_________________________________________________________________
dense_7 (Dense)              (None, 11)                561       
Total params: 10,961
Trainable params: 10,961
Non-trainable params: 0
_________________________________________________________________


**Separate data and train**

In [ ]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=11, shuffle=True)
iterations = 0 
sumOfAccuracy = 0
for train, test in kfold.split(input_data, output.argmax(1)):
  iterations += 1
  trainX, testX = input_data[train], input_data[test]
  trainY, testY = output[train], output[test]
  model.fit(trainX, trainY, epochs=15, batch_size=32, verbose=2)
  predicted = model.predict(testX)
  pred = []
  for j in predicted:
    arg = np.argmax(j)
    pred.append(arg)
  pred = np.array(pred)
  y_one = []
  for y in testY :
    arg = np.argmax(y)
    y_one.append(arg)
  y_one = np.array(y_one)
  sumOfAccuracy += acc(y_one,pred)
  print(f"average accuracy = {sumOfAccuracy/iterations}")



Epoch 1/15
5944/5944 - 22s - loss: 2.0970 - accuracy: 0.2268
Epoch 2/15
5944/5944 - 22s - loss: 1.8525 - accuracy: 0.3112
Epoch 3/15
5944/5944 - 22s - loss: 1.7456 - accuracy: 0.3491
Epoch 4/15
5944/5944 - 23s - loss: 1.6710 - accuracy: 0.3760
Epoch 5/15
5944/5944 - 22s - loss: 1.6198 - accuracy: 0.3939
Epoch 6/15
5944/5944 - 22s - loss: 1.5823 - accuracy: 0.4069
Epoch 7/15
5944/5944 - 22s - loss: 1.5524 - accuracy: 0.4179
Epoch 8/15
5944/5944 - 22s - loss: 1.5285 - accuracy: 0.4255
Epoch 9/15
5944/5944 - 22s - loss: 1.5078 - accuracy: 0.4340
Epoch 10/15
5944/5944 - 22s - loss: 1.4907 - accuracy: 0.4395
Epoch 11/15
5944/5944 - 22s - loss: 1.4766 - accuracy: 0.4445
Epoch 12/15
5944/5944 - 22s - loss: 1.4638 - accuracy: 0.4498
Epoch 13/15
5944/5944 - 22s - loss: 1.4557 - accuracy: 0.4529
Epoch 14/15
5944/5944 - 22s - loss: 1.4444 - accuracy: 0.4565
Epoch 15/15
5944/5944 - 22s - loss: 1.4366 - accuracy: 0.4598
average accuracy = 0.4585173501577287
Epoch 1/15
5944/5944 - 22s - loss: 1.4333

**Predict using trained model**

In [ ]:
prd = model.predict(X_test)

**Analyse accuracies**


In [ ]:
pred = []
for j in range(len(prd)):
  lis = prd[j]
  arg = np.argmax(lis)
  pred.append(arg)
pred = np.array(pred)

y_one = []
for y in y_test :
  arg = np.argmax(y)
  y_one.append(arg)
y_one = np.array(y_one)

print(classification_report(y_one, pred))
print(dict_id_word)
print()
print(acc(y_one,pred))

**Real life usage example**

In [ ]:
with open(f'/content/drive/MyDrive/database/silencio/11.pkl', 'rb') as input:
  readings = pickle.load(input)
  input_test_data = []
  j = 0
  for reading in readings:
    finger = []
    for i in range(5):
      finger.append(np.float16(reading.imus[i].accel[0]/255.0))
      finger.append(np.float16(reading.imus[i].accel[1]/255.0))
      finger.append(np.float16(reading.imus[i].accel[2]/255.0))
      finger.append(np.float16(reading.imus[i].gyro[0]/255.0))
      finger.append(np.float16(reading.imus[i].gyro[1]/255.0))
      finger.append(np.float16(reading.imus[i].gyro[2]/255.0))
    
  
    input_test_data.append(finger)
    
input_test_data = np.array(input_test_data)
input_test_data = input_test_data.reshape((input_test_data.shape[0], input_test_data.shape[1], 1))

for inputData in input_test_data:
  result1 = model.predict(inputData.reshape(1,30,1))
  print(dict_id_word[np.argmax(result1)])

**Plot model structure**

In [ ]:
plot_model(model, to_file='topology.png',show_shapes=True)